In [41]:
import cv2
import pandas as pd  
import os
import numpy as np 
import mediapipe as mp

In [42]:
def image_processed(file_path):
    hand_img = cv2.imread(file_path)
    # procesamos la imagen
    # 1. Convertir a RGB
    img_rgb = cv2.cvtColor(hand_img, cv2.COLOR_BGR2RGB)
    # 2. Dar la vuelta a la imagen
    img_flip = cv2.flip(img_rgb, 1)
    # 3. Mediapipe
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.7)
    output = hands.process(img_flip)
    hands.close()

    # Extraemos la información de las manos
    try:
        data = output.multi_hand_landmarks[0]
        data = str(data)
        data = data.strip().split('\n')
        garbage = ['landmark {', '  visibility: 0.0', '  presence: 0.0', '}']
        
        # Eliminamos los datos basura
        without_garbage = []
        for i in data:
            if i not in garbage:
                without_garbage.append(i)

        # Extraemos los datos de forma numérica
        clean = []
        for i in without_garbage:
            i = i.strip()
            clean.append(i[2:])

        # Eliminamos el eje z
        finalClean = []
        for i in range(0, len(clean)):
            if (i+1) % 3 != 0:
                finalClean.append(float(clean[i]))
        return(finalClean)
        
    # Si no detecta ninguna mano, devuelve un array de ceros
    except:
        return(np.zeros([1,42], dtype=int)[0])

In [43]:
file_path = 'dataset/Train/A/10.jpg'
data = image_processed(file_path)
